<a href="https://colab.research.google.com/github/Hanh42/AI---UEH-K50/blob/main/T%E1%BA%A0O_APP_G%E1%BB%8CI_XE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium
import ipywidgets as widgets
from IPython.display import display
from geopy.distance import geodesic
import random
import osmnx as ox
import networkx as nx
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [ ]:
# -*- coding: utf-8 -*-
%pip install -q osmnx folium geopy scikit-fuzzy ipywidgets

import folium
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from geopy.distance import geodesic
import random
import osmnx as ox
import networkx as nx
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# ======= 20 điểm du lịch Quận 1 =======
tourist_spots = {
    "Chợ Bến Thành": (10.7721, 106.6983),
    "Dinh Độc Lập": (10.7798, 106.6950),
    "Nhà thờ Đức Bà": (10.7797, 106.6992),
    "Bưu điện trung tâm": (10.7791, 106.6984),
    "Phố đi bộ Nguyễn Huệ": (10.7770, 106.7000),
    "Bảo tàng TP.HCM": (10.7787, 106.6950),
    "Thảo Cầm Viên Sài Gòn": (10.7790, 106.7041),
    "Bitexco Tower": (10.7711, 106.7043),
    "Cầu Mống": (10.7767, 106.7016),
    "Nhà hát TP": (10.7794, 106.6990),
    "Bảo tàng Mỹ thuật": (10.7790, 106.6981),
    "Đường sách Nguyễn Văn Bình": (10.7756, 106.7012),
    "Chợ Lớn Quận 1": (10.7742, 106.6953),
    "Ngã sáu Phù Đổng": (10.7725, 106.6995),
    "Phố Tây Bùi Viện": (10.7667, 106.6994),
    "Nhà thờ Tân Định": (10.7840, 106.6961),
    "Công viên 23/9": (10.7755, 106.6960),
    "Vincom Center": (10.7765, 106.7006),
    "Saigon Centre": (10.7733, 106.7033),
    "Bến Nhà Rồng": (10.7629, 106.6983)
}

# ======= Style header =======
display(HTML("""
<div style="background-color:#00CED1; padding:15px; text-align:center; border-radius:10px; margin-bottom:10px;">
  <h1 style="color:white; font-family:Arial Black;">RIDO</h1>
  <h3 style="color:white; font-family:Arial;">smart drive, smart life</h3>
</div>
"""))

# ======= Widgets =======
start_selector = widgets.Dropdown(options=list(tourist_spots.keys()), description='Điểm bắt đầu:')
end_selector = widgets.Dropdown(options=list(tourist_spots.keys()), description='Điểm kết thúc:')
vehicle_selector = widgets.RadioButtons(options=['Ô tô', 'Xe máy'], description='Loại phương tiện:')
distance_slider = widgets.IntSlider(value=5000, min=0, max=20000, step=100, description='Distance (m):', continuous_update=False)
traffic_slider = widgets.IntSlider(value=5, min=0, max=10, step=1, description='Traffic:', continuous_update=False)
weather_slider = widgets.IntSlider(value=5, min=0, max=10, step=1, description='Weather:', continuous_update=False)
fare_display = widgets.HTML(value="<b>Thông tin chuyến đi sẽ hiển thị ở đây</b>",
                            layout=widgets.Layout(border='2px solid purple', padding='10px', width='70%', font_size='16px', font_weight='bold'))
compute_button = widgets.Button(description="Tính tuyến đường & phí cước", button_style='success')

# ======= Fuzzy logic =======
distance = ctrl.Antecedent(np.arange(0, 20001, 1), 'distance')
traffic = ctrl.Antecedent(np.arange(0, 11, 1), 'traffic')
weather = ctrl.Antecedent(np.arange(0, 11, 1), 'weather')
fare = ctrl.Consequent(np.arange(0, 200000, 1), 'fare')

distance['short'] = fuzz.trimf(distance.universe, [0, 0, 5000])
distance['medium'] = fuzz.trimf(distance.universe, [3000, 8000, 12000])
distance['long'] = fuzz.trimf(distance.universe, [10000, 20000, 20000])
traffic['low'] = fuzz.trimf(traffic.universe, [0, 0, 5])
traffic['medium'] = fuzz.trimf(traffic.universe, [3, 5, 7])
traffic['high'] = fuzz.trimf(traffic.universe, [5, 10, 10])
weather['good'] = fuzz.trimf(weather.universe, [0, 0, 5])
weather['bad'] = fuzz.trimf(weather.universe, [3, 7, 10])
weather['severe'] = fuzz.trimf(weather.universe, [7, 10, 10])
fare['low'] = fuzz.trimf(fare.universe, [0, 0, 50000])
fare['medium'] = fuzz.trimf(fare.universe, [30000, 80000, 130000])
fare['high'] = fuzz.trimf(fare.universe, [100000, 200000, 200000])

rules = []
for d in ['short','medium','long']:
    for t in ['low','medium','high']:
        for w in ['good','bad','severe']:
            if d=='long' or t=='high' or w=='severe':
                fare_level = 'high'
            elif d=='medium' or t=='medium' or w=='bad':
                fare_level = 'medium'
            else:
                fare_level = 'low'
            rules.append(ctrl.Rule(distance[d] & traffic[t] & weather[w], fare[fare_level]))
fare_ctrl = ctrl.ControlSystem(rules)
fare_sim = ctrl.ControlSystemSimulation(fare_ctrl)

# ======= Xe ngẫu nhiên =======
def generate_random_vehicles(num_vehicles=20, center_coords=(10.775, 106.700), radius=0.01):
    vehicles = []
    for i in range(num_vehicles):
        lat_offset = random.uniform(-radius, radius)
        lon_offset = random.uniform(-radius, radius)
        new_lat = center_coords[0] + lat_offset
        new_lon = center_coords[1] + lon_offset
        vehicle_type = random.choice(["Ô tô", "Xe máy"])
        vehicles.append({'lat': new_lat, 'lon': new_lon, 'type': vehicle_type})
    return vehicles

vehicle_data = generate_random_vehicles()

# ======= Hàm tính route và hiển thị map =======
def compute_and_display(b):
    start_name = start_selector.value
    end_name = end_selector.value
    start_coords = tourist_spots[start_name]
    end_coords = tourist_spots[end_name]
    vehicle_type = vehicle_selector.value

    # Xe gần nhất
    min_distance = float('inf')
    nearest_vehicle = None
    for v in vehicle_data:
        if v['type'] == vehicle_type:
            d = geodesic(start_coords, (v['lat'], v['lon'])).meters
            if d < min_distance:
                min_distance = d
                nearest_vehicle = v

    mid_lat = (nearest_vehicle['lat'] + start_coords[0] + end_coords[0])/3
    mid_lon = (nearest_vehicle['lon'] + start_coords[1] + end_coords[1])/3
    G = ox.graph_from_point((mid_lat, mid_lon), dist=2000, network_type='drive')

    try:
        node_vehicle = ox.nearest_nodes(G, nearest_vehicle['lon'], nearest_vehicle['lat'])
        node_start = ox.nearest_nodes(G, start_coords[1], start_coords[0])
        node_end = ox.nearest_nodes(G, end_coords[1], end_coords[0])
        route_to_start = nx.shortest_path(G, node_vehicle, node_start, weight='length')
        route_start_to_end = nx.shortest_path(G, node_start, node_end, weight='length')
        coords_to_start = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in route_to_start]
        coords_start_to_end = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in route_start_to_end]
        total_distance = sum(geodesic(coords_to_start[i], coords_to_start[i+1]).meters for i in range(len(coords_to_start)-1))
        total_distance += sum(geodesic(coords_start_to_end[i], coords_start_to_end[i+1]).meters for i in range(len(coords_start_to_end)-1))
    except:
        coords_to_start = [(nearest_vehicle['lat'], nearest_vehicle['lon']), start_coords]
        coords_start_to_end = [start_coords, end_coords]
        total_distance = geodesic(coords_to_start[0], coords_to_start[1]).meters + geodesic(coords_start_to_end[0], coords_start_to_end[1]).meters

    # Fuzzy fare
    fare_sim.input['distance'] = distance_slider.value
    fare_sim.input['traffic'] = traffic_slider.value
    fare_sim.input['weather'] = weather_slider.value
    fare_sim.compute()
    base_fare = fare_sim.output['fare']
    final_fare = base_fare*0.5 if vehicle_type=='Xe máy' else base_fare

    # Thời gian (giả sử vận tốc trung bình: Ô tô 40km/h, Xe máy 30km/h)
    speed = 40 if vehicle_type=='Ô tô' else 30
    travel_time_min = total_distance/1000 / speed * 60
    wait_time_min = min_distance/1000 / speed * 60
    total_time_min = travel_time_min + wait_time_min

    fare_display.value = f"<b>Quãng đường:</b> {round(total_distance,1)} m<br>" \
                         f"<b>Phí cước:</b> {round(final_fare,0)} VND<br>" \
                         f"<b>Thời gian chờ đón:</b> {round(wait_time_min,1)} phút<br>" \
                         f"<b>Tổng thời gian chuyến đi:</b> {round(total_time_min,1)} phút"

    # Vẽ map
    m = folium.Map(location=start_coords, zoom_start=15)
    for name, coord in tourist_spots.items():
        folium.Marker(coord, popup=name, tooltip=name,
                      icon=folium.Icon(color='purple', icon='star', prefix='fa')).add_to(m)
    folium.PolyLine(coords_to_start, color='#FF5733', weight=4, opacity=0.9).add_to(m)
    folium.PolyLine(coords_start_to_end, color='#33C1FF', weight=4, opacity=0.9).add_to(m)
    for v in vehicle_data:
        color = 'blue' if v==nearest_vehicle else ('gray' if v['type']=='Ô tô' else 'red')
        folium.Marker((v['lat'], v['lon']), popup=v['type'],
                      icon=folium.Icon(color=color, icon='car' if v['type']=='Ô tô' else 'motorcycle', prefix='fa')).add_to(m)

    clear_output(wait=True)
    # Header lại
    display(HTML("""
    <div style="background-color:#00CED1; padding:15px; text-align:center; border-radius:10px; margin-bottom:10px;">
      <h1 style="color:white; font-family:Arial Black;">RIDO</h1>
      <h3 style="color:white; font-family:Arial;">smart drive, smart life</h3>
    </div>
    """))
    display(start_selector, end_selector, vehicle_selector, distance_slider, traffic_slider, weather_slider, compute_button, fare_display)
    display(m)

compute_button.on_click(compute_and_display)

# Hiển thị giao diện lần đầu
display(start_selector, end_selector, vehicle_selector, distance_slider, traffic_slider, weather_slider, compute_button, fare_display)


Dropdown(description='Điểm bắt đầu:', options=('Chợ Bến Thành', 'Dinh Độc Lập', 'Nhà thờ Đức Bà', 'Bưu điện tr…

Dropdown(description='Điểm kết thúc:', index=11, options=('Chợ Bến Thành', 'Dinh Độc Lập', 'Nhà thờ Đức Bà', '…

RadioButtons(description='Loại phương tiện:', options=('Ô tô', 'Xe máy'), value='Ô tô')

IntSlider(value=5000, continuous_update=False, description='Distance (m):', max=20000, step=100)

IntSlider(value=5, continuous_update=False, description='Traffic:', max=10)

IntSlider(value=5, continuous_update=False, description='Weather:', max=10)

Button(button_style='success', description='Tính tuyến đường & phí cước', style=ButtonStyle())

HTML(value='<b>Quãng đường:</b> 1787.1 m<br><b>Phí cước:</b> 80000.0 VND<br><b>Thời gian chờ đón:</b> 0.3 phút…